<a href="https://colab.research.google.com/github/la2015-hw/Group_10/blob/main/NaiveBayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
print("Hello world")

Hello world
